## Testing with param

Providing `model` parameters and name, running simulation to produce energy deposition vs. time plot.

In [14]:
%env ASTERIA=/home/jakob/software/ASTERIA/ASTERIA
from asteria.simulation import Simulation
import astropy.units as u

#model = {'name': 'Nakazato_2013',
#         'param':{
#             'progenitor_mass': 13*u.Msun,
#             'revival_time': 300 * u.ms,
#             'metallicity': 0.004,
#             'eos': 'shen'}
#         }

model = {'name': 'Zha_2021',
         'param':{
             'progenitor_mass': 16*u.Msun}
         }
sim = Simulation(model=model,
                 distance=10 * u.kpc, 
                 Emin=0*u.MeV, Emax=100*u.MeV, dE=1*u.MeV,
                 tmin=-0.5*u.s, tmax=1.5*u.s, dt=1*u.ms)
sim.run()

import matplotlib.pyplot as plt
for flavor in sim.flavors:
    plt.plot(sim.time, sim.E_per_V[flavor], label=flavor.name)
plt.legend()
ax = plt.gca()
ax.set(xlabel=r't-t$_{bounce}$ [s]', ylabel='Energy Deposition [Mev m$^{-3}$])', xlim=(-0.15, 0.75));

env: ASTERIA=/home/jakob/software/ASTERIA/ASTERIA


s16.dat: 3.23MiB [00:00, 61.2MiB/s]                                                                                                                                                                                


ValueError: `x` must be strictly increasing sequence.

In [3]:
from snewpy.models.ccsn import Walk_2019
from snewpy.neutrino import Flavor
from scipy.interpolate import PchipInterpolator
import numpy as np

model = Walk_2019(filename='/home/jakob/.astropy/cache/snewpy/models/Walk_2019/s40.0c_3DBH_dir1')
time = model.time

try:
    interp_lum = PchipInterpolator(x=time, y=model.luminosity[Flavor.NU_E_BAR])
except ValueError as e:
    print(e)

diff = np.diff(time)
idx = np.where(diff==0)[0]
print(f"{'time':<10s} {'idx':<5s}")
for t, i in zip(time[idx], idx):
    # Add +1 to index to point at duplicate entries
    print(f"{t.value:<10f} {i+1:<5d}") 

`x` must be strictly increasing sequence.
time       idx  
0.110000   202  
0.177700   339  
0.199450   384  
0.227050   441  
0.298840   586  
0.299840   589  
0.300840   592  
0.317840   627  
0.340850   675  
0.340850   676  
0.360000   716  
0.379800   757  
0.388780   776  
0.410860   822  
0.435360   872  
0.435360   873  
0.454740   913  
0.479720   964  
0.500000   1006 
0.521940   1051 
0.526970   1063 
0.527320   1065 
0.531420   1075 
0.560410   1134 
0.563570   1142 
0.570090   1157 


In [12]:
from snewpy.models.ccsn import Zha_2021
Zha_2021.get_param_combinations()

({'progenitor_mass': <Quantity 16. solMass>},
 {'progenitor_mass': <Quantity 17. solMass>},
 {'progenitor_mass': <Quantity 18. solMass>},
 {'progenitor_mass': <Quantity 19. solMass>},
 {'progenitor_mass': <Quantity 20. solMass>},
 {'progenitor_mass': <Quantity 21. solMass>},
 {'progenitor_mass': <Quantity 22. solMass>},
 {'progenitor_mass': <Quantity 23. solMass>},
 {'progenitor_mass': <Quantity 24. solMass>},
 {'progenitor_mass': <Quantity 25. solMass>},
 {'progenitor_mass': <Quantity 26. solMass>},
 {'progenitor_mass': <Quantity 19.89 solMass>},
 {'progenitor_mass': <Quantity 22.39 solMass>},
 {'progenitor_mass': <Quantity 30. solMass>},
 {'progenitor_mass': <Quantity 33. solMass>})

In [5]:
from snewpy._model_urls import model_urls
model_urls["Walk_2019"]

['https://github.com/SNEWS2/snewpy/raw/v1.3/models/Walk_2019/README.md',
 'https://github.com/SNEWS2/snewpy/raw/v1.3/models/Walk_2019/Walk_2019.ipynb',
 'https://github.com/SNEWS2/snewpy/raw/v1.3/models/Walk_2019/s40.0c_3DBH_dir1_LS220_nue',
 'https://github.com/SNEWS2/snewpy/raw/v1.3/models/Walk_2019/s40.0c_3DBH_dir1_LS220_nuebar',
 'https://github.com/SNEWS2/snewpy/raw/v1.3/models/Walk_2019/s40.0c_3DBH_dir1_LS220_nux']

In [ ]:
avg_dom_signal = sim.avg_dom_signal(dt=2*u.ms)
fig, ax = plt.subplots(1, figsize=(6,6))

ax.step(sim.time_binned, avg_dom_signal, where='post')

In [ ]:
from snewpy.models.ccsn import Walk_2019
Walk_2019["param"]

In [ ]:
t, detector_signal = sim.detector_hits(dt=2*u.ms)

t, i3_sig = sim.detector_hits(dt=2*u.ms, subdetector='i3')
t, dc_sig = sim.detector_hits(dt=2*u.ms, subdetector='dc')

i3_bkg = sim.detector.i3_bg(dt=2*u.ms, size=len(i3_sig))
dc_bkg = sim.detector.dc_bg(dt=2*u.ms, size=len(dc_sig))

fig, ax = plt.subplots(1,2, figsize=(10,4))
for i in range(2):
    if i == 0:
        ax[i].step(t, i3_sig, where='post', label='i3')
        ax[i].step(t, dc_sig, where='post', label='dc')
        ax[i].set_ylabel("Detector rate [Hz] (sig only)")

    elif i == 1:    
        ax[i].step(t, i3_sig+i3_bkg, where='post', label='i3')
        ax[i].step(t, dc_sig+dc_bkg, where='post', label='dc')
        ax[i].set_ylabel("Detector rate [Hz] (sig+bkg)")

    ax[i].set_xlabel("Time [s]")
    ax[i].legend()

## Testing with config file

Providing config file. 

ToDo @sgwiswold : fix errors.

In [ ]:
sim.